### Loading Required Libraries

In [1]:
from modules import PreProcessing, functions
from sqlite3 import connect 
import pandas as pd

# to ensure all changes are reflected 
# in-case jupyter notebook was already oppened

from importlib import reload
reload(PreProcessing)
reload(functions)

project_dir = '/Users/nima/repository/509_project'

### Section 1 - Data Collection

In [2]:
driver = PreProcessing.ChromeDriver(driver_path=project_dir+'/chromedriver')
scrapper = PreProcessing.WebScraper(driver)

# wait randomly between 3 to 6 seconds between each retrieval
retrieval_control = functions.RetrievalControl(min_value=3,max_value=6)

In [3]:
# define the courses which reviews needs to be collected for!

course_names = ['excel-essentials', 
                'introduction-cybersecurity-cyber-attacks', 
                'learning-how-to-learn', 
                'ai-for-everyone']

In [ ]:
for course_name in course_names:
    course = PreProcessing.Course(course_name)
    
    driver.url = course.main_page_link
    
    course.title = scrapper.extract_course_info('title')
    
    # Click on view all to expand the info section and get the full about text
    driver.click_element(by_class_name='_1wziqqgx')
    course.about_text = scrapper.extract_course_info('about_text')
        
    course.available_review_pages = scrapper.extract_course_info('available_review_pages')
    
    # if smaller portions of the reviews needs to be retrieved,
    # set what pages to tetrieve for example only get reviews from
    # page 1, 5 and 10 instead of all pages
    # course.available_review_pages = [1, 5, 10]
    
    course.save_course_info_to_file(file_location=project_dir)

    retrieval_control.wait
    
    for page_num in course.available_review_pages:

        driver.url = course.page_link(page_num)
        reviews_in_page = scrapper.extract_reviews_from_current_page()
    
        course.save_reviews_to_file(
                this_page_reviews=reviews_in_page, 
                directory=project_dir + '/reviews/')
        
        retrieval_control.wait

### Section 2 - Gather all data into a database
This will help with performing queries using sql statements <br>
as well as sharing the data with others in a single file

In [8]:
databse_file = 'database.db'

course_info = pd.read_csv(project_dir + '/course_info.csv')
review_files = os.listdir(project_dir + '/reviews/')

all_reviews = pd.DataFrame()

for file_path in review_files:
    course_review = pd.read_csv(project_dir + '/reviews/' + file_path)
    course_name = file_path.replace('.csv','')
    course_review['course_name'] = course_name
    all_reviews = pd.concat([all_reviews, course_review], ignore_index=True)
  
conn = connect(':memory:')
all_reviews.to_sql('reviews', conn)
course_info.to_sql('course', conn)


if os.path.exists(databse_file):
    print('Database already exists!\n',
          'change %s to something else.\n' % databse_file,
          'This is to avoid loosing data from before!')
else:
    main = connect(databse_file)
    conn.backup(main)
    main.commit()
    main.close()

Database already exists!
 change database.db to something else.
 This is to avoid loosing data from before!


In case if the data has already been converted to a database, <br>
the block above can be skippled and only load the database

In [10]:
# test if all data can be accessed.
main = connect(databse_file)

query = \
"""
SELECT course_name, text 
FROM reviews 
WHERE helpful > 20 
LIMIT 5
"""

pd.read_sql_query(query, main)

,course_name,text
0,excel-essentials,This course is simply awesome. If you really w...
1,excel-essentials,I have working experience with Excel for many ...
2,excel-essentials,Practice make permanent is the true line .this...
3,excel-essentials,A great course for beginners in MS Excel. Intr...
4,excel-essentials,This is an amazing course for learners. I am h...
